In [1]:
from pathlib import Path
import os

In [2]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.readers.file import PDFReader, PyMuPDFReader
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.storage.index_store.redis import RedisIndexStore
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.core.node_parser import SentenceSplitter

from redis import Redis
from redisvl.schema import IndexSchema

In [4]:
REDIS_HOST = os.getenv("REDIS_HOST", "127.0.0.1")
REDIS_PORT = os.getenv("REDIS_PORT", 6379)

In [5]:
endpoint = "http://phoenix:6006/v1/traces"  # Phoenix receiver address

tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [6]:
file_details = {
    "2Q24 Earnings Release_Final.pdf": "2nd quarter 2024 earnings release Final of PNC Bank with detailed financial information for first two quarters of year 2024.",
    "2Q24 Financial Supplement_Final.pdf": "Supplemental data for 2nd quarter 2024 earnings release Final of PNC Bank with financial tables.",
    "Board of Directors  PNC.pdf": "PNC Bank Board of Directors member information.",
    "PNC 2023 10-K.pdf": "PNC Bank Form 10-K report for year 2023.",
    "PNC 2023 Annual Report.pdf": "PNC Bank detailed Annual Report for year 2023.",
    "pnc_privacy_notice.pdf": "Information on what kind of customer personal information PNC Bank share and not.",
}
def get_meta(filename):
    return {
        "file_path": filename,
        "file_details": file_details.get(
            Path(filename).name, ""
        ),
    }

In [7]:
custom_schema = IndexSchema.from_dict(
    {
        # customize basic index specs
        "index": {
            "name": "user-index",
            "prefix": "index",
            "key_separator": ":",
        },
        # customize fields that are indexed
        "fields": [
            # required fields for llamaindex
            {"type": "tag", "name": "id"},
            {"type": "tag", "name": "doc_id"},
            {"type": "text", "name": "text"},
            # custom metadata fields
            {"type": "numeric", "name": "updated_at"},
            {"type": "tag", "name": "file_name"},
            # custom vector field definition for cohere embeddings
            {
                "type": "vector",
                "name": "vector",
                "attrs": {
                    "dims": 768,
                    "algorithm": "hnsw",
                    "distance_metric": "cosine",
                },
            },
        ],
    }
)

In [8]:
storage_context = StorageContext.from_defaults(
    docstore=RedisDocumentStore.from_host_and_port(
        host=REDIS_HOST, port=REDIS_PORT, namespace="llama_index"
    ),
    index_store=RedisIndexStore.from_host_and_port(
        host=REDIS_HOST, port=REDIS_PORT, namespace="llama_index"
    ),
    vector_store=RedisVectorStore(redis_client=Redis.from_url(f"redis://{REDIS_HOST}:{REDIS_PORT}"), 
                                  overwrite=True,
                                  schema=custom_schema
                                  )
)

00:21:14 redisvl.index.index INFO   Index already exists, overwriting.


In [9]:
# PDF Reader with `SimpleDirectoryReader`
parser = PyMuPDFReader()
file_extractor = {".pdf": parser}

In [10]:
documents = SimpleDirectoryReader(
    "./pnc", file_metadata=get_meta, file_extractor=file_extractor
).load_data()

In [11]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

00:21:16 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

00:21:23 sentence_transformers.SentenceTransformer INFO   2 prompts are loaded, with the keys: ['query', 'text']


In [12]:
# Groq
Settings.llm = Groq(
    model="llama-3.1-8b-instant"
)

In [13]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:33 llama_index.vector_stores.redis.base INFO   Added 712 documents to index user-index


In [14]:
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query(
    "Who is PNC Chairman? Provide evidence from documents to support your answer."
)
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:33 llama_index.vector_stores.redis.base INFO   Querying index user-index with filters *
00:27:33 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['index:efc92741-d1ab-4457-94de-3fbdbcf91c26', 'index:82b92b51-a111-4c64-8c84-e9d75fb90b58']
00:27:34 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:27:34 openinference.instrumentation.llama_index._handler ERROR   Error serializing to JSON: TypeError: 'MockValSer' object cannot be converted to 'SchemaSerializer'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openinference/instrumentation/llama_index/_handler.py", line 248, in process_output
    self[OUTPUT_VALUE] = result.model_dump_json(exclude_unset=True)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pydantic/main.py", line 415, in model_dump_json
    return self.__pydantic_serializer__.to_json(
   

In [16]:
response = query_engine.query("Tell me about Bryan Salesky work experiance.")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:34 llama_index.vector_stores.redis.base INFO   Querying index user-index with filters *
00:27:34 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['index:fa1dcf8a-f5a9-4320-8ab0-9c95db4fa20a', 'index:0e874657-fa40-4313-a64b-6897e0dde9df']
00:27:35 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:27:35 openinference.instrumentation.llama_index._handler ERROR   Error serializing to JSON: TypeError: 'MockValSer' object cannot be converted to 'SchemaSerializer'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openinference/instrumentation/llama_index/_handler.py", line 248, in process_output
    self[OUTPUT_VALUE] = result.model_dump_json(exclude_unset=True)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pydantic/main.py", line 415, in model_dump_json
    return self.__pydantic_serializer__.to_json(
   

In [17]:
response = query_engine.query("What does PNC do with customer personal information?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:35 llama_index.vector_stores.redis.base INFO   Querying index user-index with filters *
00:27:35 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['index:167080bf-d5a9-4838-92f8-8eae935131fa', 'index:cd4e30d1-43b6-46d3-8901-861ceb363d4a']
00:27:36 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:27:36 openinference.instrumentation.llama_index._handler ERROR   Error serializing to JSON: TypeError: 'MockValSer' object cannot be converted to 'SchemaSerializer'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openinference/instrumentation/llama_index/_handler.py", line 248, in process_output
    self[OUTPUT_VALUE] = result.model_dump_json(exclude_unset=True)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pydantic/main.py", line 415, in model_dump_json
    return self.__pydantic_serializer__.to_json(
   

In [18]:
response = query_engine.query(
    "What was PNC diluted earnings per common share in 2023? Provide evidence from documents to support your answer."
)
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:36 llama_index.vector_stores.redis.base INFO   Querying index user-index with filters *
00:27:36 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['index:6bcfe318-32d8-47fe-bad2-def27e1d17b1', 'index:5a1c3174-c8cd-4994-9067-0c8266ea108b']
00:27:36 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:27:36 openinference.instrumentation.llama_index._handler ERROR   Error serializing to JSON: TypeError: 'MockValSer' object cannot be converted to 'SchemaSerializer'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openinference/instrumentation/llama_index/_handler.py", line 248, in process_output
    self[OUTPUT_VALUE] = result.model_dump_json(exclude_unset=True)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pydantic/main.py", line 415, in model_dump_json
    return self.__pydantic_serializer__.to_json(
   

In [19]:
response = query_engine.query(
    "How good PNC revenue was in 2023 compared to previous years?"
)
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:37 llama_index.vector_stores.redis.base INFO   Querying index user-index with filters *
00:27:37 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['index:52429ce2-7726-4c40-884d-9d9662874121', 'index:47d4b26a-330b-47a4-9bf1-3b5163c78021']
00:27:37 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:27:37 openinference.instrumentation.llama_index._handler ERROR   Error serializing to JSON: TypeError: 'MockValSer' object cannot be converted to 'SchemaSerializer'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openinference/instrumentation/llama_index/_handler.py", line 248, in process_output
    self[OUTPUT_VALUE] = result.model_dump_json(exclude_unset=True)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pydantic/main.py", line 415, in model_dump_json
    return self.__pydantic_serializer__.to_json(
   

In [20]:
response = query_engine.query("Can you print a table showing PNC Revenue, Net Income and Total Non-interst Expenses for first two quarters of 2024? Provide evidence from documents to support your answer.")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

00:27:37 llama_index.vector_stores.redis.base INFO   Querying index user-index with filters *
00:27:37 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['index:c57fc9fb-d1fc-42df-9879-f64c4d70cbec', 'index:b6af0711-799f-4e2b-9651-bf4580e90ff3']
00:27:38 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
00:27:38 openinference.instrumentation.llama_index._handler ERROR   Error serializing to JSON: TypeError: 'MockValSer' object cannot be converted to 'SchemaSerializer'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openinference/instrumentation/llama_index/_handler.py", line 248, in process_output
    self[OUTPUT_VALUE] = result.model_dump_json(exclude_unset=True)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pydantic/main.py", line 415, in model_dump_json
    return self.__pydantic_serializer__.to_json(
   